## A simple self-attention mechanism without trainable weights

In [ ]:
import torch
inputs = torch.tensor(
[[0.43, 0.15, 0.89], # Your
[0.55, 0.87, 0.66], # journey
[0.57, 0.85, 0.64], # starts
[0.22, 0.58, 0.33], # with
[0.77, 0.25, 0.10], # one
[0.05, 0.80, 0.55]] # step
)

In [ ]:
query = inputs[1]

attention_scores = torch.empty(inputs.shape[0])

for i, x_i in enumerate(inputs):
  attention_scores[i] = torch.dot(x_i, query)

print(attention_scores)

tensor([0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865])


In [ ]:
attention_scores_tmp = attention_scores / attention_scores.sum()
print(attention_scores)

tensor([0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865])


In [ ]:
import numpy as np

def naive_softmax(x):
  return np.exp(x) / np.exp(x).sum(dim=0)

print(naive_softmax(attention_scores))

tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])


In [ ]:
attention_scores_normalized = torch.softmax(attention_scores, dim=0)

In [ ]:
attention_scores_normalized

tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])

In [ ]:
query = inputs[1]
context_vector = torch.zeros(query.shape)
for i, x_i in enumerate(inputs):
  context_vector += attention_scores_normalized[i] * x_i

print(context_vector)

tensor([0.4419, 0.6515, 0.5683])


## Computing attention weights for all input tokens

In [ ]:
att_scores = torch.empty(6,6)

for i, x_i in enumerate(inputs):
  for j, x_j in enumerate(inputs):
    att_scores[i,j] = torch.dot(x_i, x_j)

print(att_scores)

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])


In [ ]:
attn_scores = inputs @ inputs.T

attn_scores

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])

In [ ]:
attn_weights = torch.softmax(attn_scores, dim=-1)

attn_weights

tensor([[0.2098, 0.2006, 0.1981, 0.1242, 0.1220, 0.1452],
        [0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581],
        [0.1390, 0.2369, 0.2326, 0.1242, 0.1108, 0.1565],
        [0.1435, 0.2074, 0.2046, 0.1462, 0.1263, 0.1720],
        [0.1526, 0.1958, 0.1975, 0.1367, 0.1879, 0.1295],
        [0.1385, 0.2184, 0.2128, 0.1420, 0.0988, 0.1896]])

In [ ]:
attn_weights[1].sum()

tensor(1.)

In [ ]:
all_context_vecs = attn_weights @ inputs

all_context_vecs

tensor([[0.4421, 0.5931, 0.5790],
        [0.4419, 0.6515, 0.5683],
        [0.4431, 0.6496, 0.5671],
        [0.4304, 0.6298, 0.5510],
        [0.4671, 0.5910, 0.5266],
        [0.4177, 0.6503, 0.5645]])

# Implementing self-attention with trainable weights

## Computing the attention weights step by step

In [ ]:
x_2 = inputs[1]
d_in = inputs.shape[1]
d_out = 2

In [ ]:
torch.manual_seed(42)

W_query = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)
W_key = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)
W_value = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)

In [ ]:
query_2 = x_2 @ W_query
key_2 = x_2 @ W_key
value_2 = x_2 @ W_value

print(query_2)
print(key_2)
print(value_2)

tensor([1.0760, 1.7344])
tensor([1.5764, 0.9441])
tensor([1.7073, 1.0646])


In [ ]:
keys = inputs @ W_key
values = inputs @ W_value

print(keys)
print(values)

tensor([[1.0832, 0.8895],
        [1.5764, 0.9441],
        [1.5440, 0.9455],
        [0.9105, 0.4477],
        [0.5262, 0.7038],
        [1.2795, 0.4727]])
tensor([[1.2731, 0.8193],
        [1.7073, 1.0646],
        [1.6922, 1.0559],
        [0.9133, 0.5633],
        [0.9433, 0.6019],
        [1.1233, 0.6876]])


In [ ]:
keys_2 = keys[1]
attn_score_2 = keys_2.dot(query_2)

attn_score_2

tensor(3.3338)

In [ ]:
attn_scores_2 = query_2 @ keys.T

attn_scores_2

tensor([2.7084, 3.3338, 3.3013, 1.7563, 1.7869, 2.1966])

In [ ]:
d_k = keys.shape[1]

attn_weights_2 = torch.softmax(attn_scores_2 / (d_k ** 0.5), dim=-1)

attn_weights_2

tensor([0.1723, 0.2681, 0.2620, 0.0879, 0.0898, 0.1200])

In [ ]:
context_vect_2 = attn_weights_2 @ values

context_vect_2

tensor([1.4201, 0.8892])

## Implementing a compact self-attention Python class

In [ ]:
import torch.nn as nn

class SelfAttention_v1(nn.Module):
  def __init__(self, d_in, d_out):
    super().__init__()
    self.w_key = nn.Parameter(torch.rand(d_in, d_out))
    self.w_query = nn.Parameter(torch.rand(d_in, d_out))
    self.w_value = nn.Parameter(torch.rand(d_in, d_out))


  def forward(self, x):
    keys = x @ self.w_key
    queries = x @ self.w_query
    values = x @ self.w_value

    attn_scores = queries @ keys.T
    attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)
    context_vec = attn_weights @ values

    return context_vec

In [ ]:
torch.manual_seed(42)
sa_v1 = SelfAttention_v1(d_in, d_out)
print(sa_v1(inputs))

tensor([[1.3916, 0.8710],
        [1.4175, 0.8872],
        [1.4162, 0.8864],
        [1.3492, 0.8448],
        [1.3519, 0.8462],
        [1.3676, 0.8564]], grad_fn=<MmBackward0>)


In [ ]:
class SelfAttention_v2(nn.Module):
  def __init__(self, d_in, d_out, kqv_bias=False):
    super().__init__()
    self.W_query = nn.Linear(d_in, d_out, bias=kqv_bias)
    self.W_key = nn.Linear(d_in, d_out, bias=kqv_bias)
    self.W_value = nn.Linear(d_in, d_out, bias=kqv_bias)

  def forward(self, x):
    keys = self.W_key(x)
    queries = self.W_query(x)
    values = self.W_value(x)

    attn_scores = queries @ keys.T
    attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)
    context_vect = attn_weights @ values
    return context_vect

In [ ]:
torch.manual_seed(42)
sa_v2 = SelfAttention_v2(d_in, d_out)
print(sa_v2(inputs))

tensor([[0.3755, 0.2777],
        [0.3761, 0.2831],
        [0.3761, 0.2833],
        [0.3768, 0.2763],
        [0.3754, 0.2836],
        [0.3772, 0.2746]], grad_fn=<MmBackward0>)


# Hiding future words with causal attention

## Applying a causal attention mask

In [ ]:
queries = sa_v2.W_query(inputs)
keys = sa_v2.W_key(inputs)
values = sa_v2.W_value(inputs)

attn_scores = queries @ keys.T

attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)

print(attn_weights)

tensor([[0.1605, 0.1726, 0.1714, 0.1681, 0.1473, 0.1801],
        [0.1627, 0.1780, 0.1758, 0.1648, 0.1306, 0.1880],
        [0.1625, 0.1782, 0.1759, 0.1648, 0.1302, 0.1885],
        [0.1661, 0.1726, 0.1715, 0.1654, 0.1475, 0.1768],
        [0.1596, 0.1777, 0.1755, 0.1664, 0.1312, 0.1896],
        [0.1682, 0.1715, 0.1707, 0.1648, 0.1511, 0.1738]],
       grad_fn=<SoftmaxBackward0>)


In [ ]:
context_length = attn_scores.shape[0]
mask_simple = torch.tril(torch.ones(context_length, context_length))
print(mask_simple)

tensor([[1., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1.]])


In [ ]:
masked_simple = attn_weights*mask_simple
print(masked_simple)

tensor([[0.1605, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1627, 0.1780, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1625, 0.1782, 0.1759, 0.0000, 0.0000, 0.0000],
        [0.1661, 0.1726, 0.1715, 0.1654, 0.0000, 0.0000],
        [0.1596, 0.1777, 0.1755, 0.1664, 0.1312, 0.0000],
        [0.1682, 0.1715, 0.1707, 0.1648, 0.1511, 0.1738]],
       grad_fn=<MulBackward0>)


In [ ]:
row_sums = masked_simple.sum(dim=-1, keepdim=True)
masked_simple_norm = masked_simple / row_sums
print(masked_simple_norm)

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.4775, 0.5225, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3146, 0.3450, 0.3405, 0.0000, 0.0000, 0.0000],
        [0.2459, 0.2555, 0.2538, 0.2448, 0.0000, 0.0000],
        [0.1969, 0.2193, 0.2165, 0.2053, 0.1619, 0.0000],
        [0.1682, 0.1715, 0.1707, 0.1648, 0.1511, 0.1738]],
       grad_fn=<DivBackward0>)


In [ ]:
mask = torch.triu(torch.ones(context_length, context_length), diagonal=1)
masked = attn_scores.masked_fill(mask.bool(), -torch.inf)
print(masked)

tensor([[ 0.0508,    -inf,    -inf,    -inf,    -inf,    -inf],
        [ 0.2157,  0.3428,    -inf,    -inf,    -inf,    -inf],
        [ 0.2163,  0.3467,  0.3282,    -inf,    -inf,    -inf],
        [ 0.1257,  0.1799,  0.1707,  0.1191,    -inf,    -inf],
        [ 0.1667,  0.3193,  0.3012,  0.2258, -0.1098,    -inf],
        [ 0.1269,  0.1548,  0.1475,  0.0978, -0.0247,  0.1731]],
       grad_fn=<MaskedFillBackward0>)


In [ ]:
attn_weights = torch.softmax(masked / keys.shape[-1]**0.5, dim=1)
print(attn_weights)

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.4775, 0.5225, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3146, 0.3450, 0.3405, 0.0000, 0.0000, 0.0000],
        [0.2459, 0.2555, 0.2538, 0.2448, 0.0000, 0.0000],
        [0.1969, 0.2193, 0.2165, 0.2053, 0.1619, 0.0000],
        [0.1682, 0.1715, 0.1707, 0.1648, 0.1511, 0.1738]],
       grad_fn=<SoftmaxBackward0>)


## Masking additional attention weights with dropout

In [ ]:
torch.manual_seed(123)
dropout = torch.nn.Dropout(0.5)
example = torch.ones(6, 6)
print(dropout(example))

tensor([[2., 2., 2., 2., 2., 2.],
        [0., 2., 0., 0., 0., 0.],
        [0., 0., 2., 0., 2., 0.],
        [2., 2., 0., 0., 0., 2.],
        [2., 0., 0., 0., 0., 2.],
        [0., 2., 0., 0., 0., 0.]])


In [ ]:
torch.manual_seed(123)
print(dropout(attn_weights))

tensor([[2.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 1.0449, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.6809, 0.0000, 0.0000, 0.0000],
        [0.4918, 0.5110, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3938, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.3431, 0.0000, 0.0000, 0.0000, 0.0000]],
       grad_fn=<MulBackward0>)


## Implementing a compact causal attention class

In [ ]:
batch = torch.stack((inputs, inputs), dim=0)
print(batch.shape)

torch.Size([2, 6, 3])


In [ ]:
class CausalAttention(nn.Module):
  def __init__(self, d_in, d_out, context_length, dropout=0.0, kqv_bias=False):
    super().__init__()
    self.d_out = d_out
    self.W_query = nn.Linear(d_in, d_out, bias=kqv_bias)
    self.W_key = nn.Linear(d_in, d_out, bias=kqv_bias)
    self.W_value = nn.Linear(d_in, d_out, bias=kqv_bias)
    self.dropout = nn.Dropout(dropout)
    self.register_buffer('mask', torch.triu(torch.ones(context_length, context_length), diagonal=1))

  def forward(self, x):
    b, num_tokens, d_in = x.shape
    keys = self.W_key(x)
    values = self.W_value(x)
    queries = self.W_query(x)
    attn_scores = queries @ keys.transpose(1, 2)
    attn_scores.masked_fill_(
        self.mask.bool()[:num_tokens, :num_tokens], -torch.inf
    )
    attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)
    attn_weights = self.dropout(attn_weights)
    context_vect = attn_weights @ values
    return context_vect

In [ ]:
torch.manual_seed(123)
ca = CausalAttention(d_in, d_out, context_length=batch.shape[1], dropout=0.0)
context_vecs = ca(batch)
print(context_vecs)

tensor([[[-0.4519,  0.2216],
         [-0.5874,  0.0058],
         [-0.6300, -0.0632],
         [-0.5675, -0.0843],
         [-0.5526, -0.0981],
         [-0.5299, -0.1081]],

        [[-0.4519,  0.2216],
         [-0.5874,  0.0058],
         [-0.6300, -0.0632],
         [-0.5675, -0.0843],
         [-0.5526, -0.0981],
         [-0.5299, -0.1081]]], grad_fn=<UnsafeViewBackward0>)


## A wrapper class to implement multi-head attention

In [1]:
import torch.nn as nn

class MultiHeadAtentionWrapper(nn.Module):
  def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias=False):
    super().__init__()
    self.heads = nn.ModuleList([
        CausalAttention(d_in, d_out, dropout, qkv_bias) for _ in range(num_heads)
    ])

  def forward(self, x):
    return torch.cat([head(x) for head in self.heads], dim=-1)

# An efficient multi-head attention class

In [2]:
class MultiHeadAttention(nn.Module):
  def __init__(self, d_in, d_out, context_length, dropout, num_heads, kqv_bias=False):
    super().__init__()
    assert (d_out%num_heads==0), "d_out should be divisable by num_heads"
    self.d_out = d_out
    self.num_heads = num_heads
    self.head_dim = d_out//num_heads
    self.w_key = nn.Linear(d_in, d_out, bias=kqv_bias)
    self.w_query = nn.Linear(d_in, d_out, bias=kqv_bias)
    self.w_value = nn.Linear(d_in, d_out, bias=kqv_bias)
    self.out_proj = nn.Linear(d_out, d_out)
    self.dropout = nn.Dropout(dropout)
    self.register_buffer(
        "triu",
        torch.triu(torch.ones(context_length, context_length), diagonal=1)
    )

  def forward(self, x):
    b, num_tokens, d_in = x.shape
    keys = self.w_key(x)
    values = self.w_value(x)
    queries = self.w_query(x)
    keys = keys.view(b, num_tokens, self.num_heads, self.head_dim)
    values = values.view(b, num_tokens, self.num_heads, self.head_dim)
    queries = queries.view(b, num_tokens, self.num_heads, self.head_dim)

    keys = keys.transpose(1,2)
    values = values.transpose(1,2)
    queries = queries.transpose(1,2)

    attn_scores = queries @ keys.transpose(2,3)
    mask_bool = self.mask.bool()[:num_tokens, :num_tokens]

    attn_scores.masked_fill_(mask_bool, -torch.inf)

    attn_weights = torch.softmax(attn_scores / keys.shape[1]**0.5, dim=-1)
    attn_weigths = self.dropout(attn_weights)

    context_vec = context_vec.contigous().view(b, num_tokens, self.d_out)
    context_vec = self.out_proj(context_vec)

    return context_vec